<a href="https://colab.research.google.com/github/Tessellate-Imaging/monk_v1/blob/master/tutorials/7_Gender_Age_Classification(Pytorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Application



Goal : Gender and age classification


## Setup Monk

#### Download Monk

In [ ]:
!git clone https://github.com/Tessellate-Imaging/monk_v1

#### Setup dependencies

In [ ]:
cd monk_v1

In [ ]:
!ls installation

In [ ]:
!pip3 install -r ./installation/requirements_cu10.txt

In [ ]:
cd ..

#### Import Monk

In [ ]:
import os
import sys
sys.path.append("./monk_v1/monk");
import psutil
from pytorch_prototype import prototype

## Prepare Dataset

#### Download Dataset

In [ ]:
!wget https://www.dropbox.com/s/3c34o03agjuydlg/age.zip

#### Extract Dataset

In [ ]:
#!rm -rf age

In [ ]:
%%capture
!unzip age.zip

#### Create Gender dataset

In [ ]:
!mkdir gender
!mkdir gender/male
!mkdir gender/female

In [ ]:
import glob
import shutil
from tqdm import tqdm


maleAgeList = glob.glob("./age/male/*")
femaleAgeList = glob.glob("./age/female/*")

for i in tqdm(maleAgeList):
  imageList = glob.glob(i + "/*.png")
  for j in imageList:
    outPath = "./gender/male/" + j.split('/')[-1]
    shutil.move(j,outPath)

for i in tqdm(femaleAgeList):
  imageList = glob.glob(i + "/*.jpg")
  for j in imageList:
    outPath = "./gender/female/" + j.split('/')[-1]
    shutil.move(j,outPath)


## Gender Classification Training

#### Create experiment

In [ ]:
ptf = prototype(verbose=1);
ptf.Prototype("Gender_Classification", "exp-1");

#### List Models to Choose from

In [ ]:
ptf.List_Models()

#### Load Dataset

In [ ]:
ptf.Default(dataset_path="./gender/", model_name="resnet18", freeze_base_network=True, num_epochs=4)

In [ ]:
ptf.Summary()

#### Break and Resume training

In [ ]:
#ptf = prototype(verbose=1);
#ptf.Prototype("Gender_Classification", "exp-1", resume_train=True);

#### Train

In [ ]:
ptf.Train()

#### Zip and download workspace

In [ ]:
#!zip -r workspace.zip workspace

In [ ]:
#from google.colab import files
#files.download("workspace.zip")

## Age Classification Male

#### Create Experiment

In [ ]:
ptf = prototype(verbose=1);
ptf.Prototype("Age_Classification", "male2");

In [ ]:
ptf.Default(dataset_path="./age/male/", model_name="densenet169", freeze_base_network=True, num_epochs=5)

#### Train

In [ ]:
%%capture
ptf.Train()

## Age Classification Female

#### Create Experiment

In [ ]:
ptf = prototype(verbose=1);
ptf.Prototype("Age_Classification", "female");

In [ ]:
ptf.Default(dataset_path="./age/male/", model_name="resnet18", freeze_base_network=True, num_epochs=5)

#### Train

In [ ]:
ptf.Train()

## Inference

#### Get the image

In [ ]:
import requests
test_url = "https://i2-prod.mirror.co.uk/incoming/article10883649.ece/ALTERNATES/s615b/PROD-Idris-Elba.jpg"

# URL of the image to be downloaded is defined as image_url 
r = requests.get(test_url) # create HTTP response object 
with open('test.jpg','wb') as f: 
    f.write(r.content) 

#### Set experiment to Inference

In [ ]:
ptf.Prototype("Gender_Classification", "exp-1", eval_infer=True);

#### Run predictions

In [ ]:
img_name = "test.jpg";
predictions = ptf.Infer(img_name=img_name, return_raw=True);
print(predictions)